In [1]:
#ここでは３次カンドルホモロジーをSmith normal formを用いて計算してみようと思います

In [2]:
from tqdm import tqdm

In [2]:
import itertools
import numpy as np

In [3]:
for i in tqdm(range(1000000000)):
    a=1

100%|█████████████████████████████████████████████████████████████| 1000000000/1000000000 [09:46<00:00, 1706049.69it/s]


In [5]:
import pdb

In [50]:
#位数３のダイヘドラルカンドル
Q = [0,1,2,]
def mul(x,y):
    return (-x+2*y)%3
def mulinv(y,z):
    return (2*y-z)%3

In [51]:
#ここからSmithnormalformを返す関数を実装していきます
#整数行列Aに対してSBTを返したい
#ほとんど潰れるはずだからSmithnormalformは基本的には1ばかりのはずよ



def Smithnormalform(A):
    SmithA = []
    #AのSmithnormalformにしたいやつ
    S = np.eye(len(A))
    T = np.eye(len(A[0]))
    #ここでt回目（0からlen(A[0])まで）のアルゴリズムを挟む
    for t in range(0,len(A[0])):
        algoend = True
        while algoend:
            algoend = Smithalgo(A,t)[3]
            S = np.dot(Smithalgo(A,t)[0],S)
            T = np.dot(T,Smithalgo(A,t)[2])
            A = Smithalgo(A,t)[1]
       # print(A)
       # print(None)
    
    #ここからalgoのfinalstepだよ
    S = np.dot(Smithalgofinalstep(A)[0],np.array(S))
    T = np.dot(np.array(T),Smithalgofinalstep(A)[2])
    A = Smithalgofinalstep(A)[1]
    
    SmithA = [S,A,T]
    return SmithA
#algofinalstepの直前までは上手くいってます



def Smithalgo(A,t):
    S = np.eye(len(A))
    T = np.eye(len(A[0]))
    #tは何列目に対してアルゴを回すか
    jt = 0
    #t列目のjt行がpivotのある場所
    jtkari = 0
    ajtt = 0
    #pivotの値にする予定
    #A[jt]がpivot行
    for v in A:
        if abs(v[t]) == 1:
            ajtt=v[t]
            jt = jtkari
            break
        elif v[t]!=0:
            if ajtt==0:
                ajtt = v[t]
                jt = jtkari
            else:
                if abs(v[t])<abs(ajtt):
                    ajtt = v[t]
                    jt = jtkari
        jtkari += 1
        #jtkariは最後まで行った場合はlen(A)となることに注意
    #ここまでで大体step1,pivotの場所を指定している
    
    
    if ajtt != 0:
        #つまりどこかに0でない元があれば
        check = False
        while check == False:
            k = Smithalgostep2(A,t,jt)
            A = k[0]
            S = np.dot(np.array(k[1]),np.array(S))
            check = k[2]
        #列を省き終わった
        

    A2 = A.T
    #転置に対しても同じことをする
    #上のAをA2に書き換えただけ
    
    jt = 0
    #t列目のjt行がpivotのある場所
    jtkari = 0
    ajtt = 0
    #pivotの値にする予定
    #A[jt]がpivot行
    for v in A2:
        if abs(v[t]) == 1:
            ajtt=v[t]
            jt = jtkari
            break
        elif v[t]!=0:
            if ajtt==0:
                ajtt = v[t]
                jt = jtkari
            else:
                if abs(v[t])<abs(ajtt):
                    ajtt = v[t]
                    jt = jtkari
        jtkari += 1
    #ここまでで大体step1,pivotの場所を指定した
    
    if ajtt != 0:
        check = False
        while check == False:
            k = Smithalgostep2(A2,t,jt)
            A2 = k[0]
            T = np.dot(np.array(k[1]),np.array(T))
            check = k[2]
    #転置した後の列を省き終わった
    A = A2.T
    T = T.T
    
    algoend = True
    if t==jt or jtkari == len(A[0]):
        #t行をt,t要素で省いたかt行が全部0だった
        habukicheck = True
        for i in range(0,len(A)):
            if i!=t and A[i][t]!=0:
                habukicheck = False
                break
        if habukicheck:
            #t列目も綺麗に省けていた場合
            algoend = False
    #algoendがFalseならt,t要素できれいに行と列を省いてます
    
    
    return [S,A,T,algoend]
#行列Aのt行目を最小の値で省いてからt列目で同様の操作、algoend==Falseならきれいに省き終わった。
#d3の場合はS,Tがうまくいきました。あってる





def Smithalgostep2(A,t,jt):
    #Aのt列目を消したいjt行目が一番絶対値小さい
    S = np.eye(len(A))
    #左からかける行列SとA自体を弄って返す
#    vpivot = []
#    for a in A[jt]:
#        vpivot.append(a)
#    #A自身をいじったのちに必要
    
    pivotcheck = True
    #ajttでt列目が割れるかどうか判定
    kt=0
    #割り切れなかった時の割れなかった行数
    for v in A:
        if v[t]%A[jt][t]!=0:
            pivotcheck = False
            break
        kt+=1
        
    if pivotcheck:
    #割り切れたとき
        for k in range(0,len(A)):
            if k!=jt:
                S[k][jt] = -A[k][t]/A[jt][t]
                #jt行目でk行を消す
        S2 = np.eye(len(A))
        if t!=jt:
            S2[t][t]=0
            S2[jt][t]=1
            S2[t][jt]=1 
            S2[jt][jt]=0
        S = np.dot(np.array(S2),np.array(S))
        #もしt!=jtならjt行目をt行目と入れ替える
        A = np.dot(np.array(S),np.array(A))
    
    else:
        #割り切れなかった時
        #A[kt][t]とA[jt][t]で最大公約数云々をする
        beta = gcd(A[kt][t],A[jt][t])
        #本当はここから拡張ユークリッド互除法でσ、τを探したほうがいいよ
        for i1 in range(int(-abs(A[jt][t]/beta)+1),int(abs(A[jt][t])/beta)):
            for i2 in range(int(-abs(A[kt][t]/beta)+1),int(abs(A[kt][t])/beta)):
                sagasicheck = False
                if A[kt][t]*i1+A[jt][t]*i2==beta:
                    S[jt][jt]=i2
                    S[kt][jt]=i1
                    S[jt][kt]=-int(A[kt][t]/beta)
                    S[kt][kt]=int(A[jt][t]/beta)
                    sagasicheck = True
                    break
            if sagasicheck:
                break
        A = np.dot(np.array(S),np.array(A))
    return [A,S,pivotcheck]
#全部割り切れるなら（pivotcheck==True）Aのt列目を省いてjt行目をt行目に移動する。この時右からSをかけてる
#全部割れないなら（pivotcheck==False）Aのjt行目を少し小さくして返す。この時右からSをかけてる
#Smithalgoが平気ゆえこれも大丈夫そう
    
def gcd(m,n):
#最大公約数とその公約数を実現する係数σとτを返します                    
    x = max(abs(m),abs(n))
    y = min(abs(m),abs(n))
    if x%y==0:
        return y
    #今回は必ずｍの絶対値が大きい
    else:
        while x%y!=0:
            z=x%y
            x=y
            y=z
        else:
            return z

        
        
        
def Smithalgofinalstep(A):
    S = np.eye(len(A))
    T = np.eye(len(A[0]))
    check = True
    while check:
        S = np.dot(sort(A)[0],np.array(S))
        T = np.dot(np.array(T),sort(A)[2])
        A = sort(A)[1]
        check = modifydiag(A)[3]
        S = np.dot(modifydiag(A)[0],np.array(S))
        T = np.dot(np.array(T),modifydiag(A)[2])
        A = modifydiag(A)[1]
#        print(check)
#        print(A.astype(np.int))
#        ここでSmithnormalformは3回Smithalgofinalstepが入っているよ
    return [S,A,T]
#Aの対角線成分の最大公倍数以上の値が最後に来る
#よって与えるAによっては簡単に浮動小数点表記が必要になります。





def sort(A):
    diag = []
    for i in range(0,len(A[0])):
        diag.append(abs(A[i][i]))
#    print(diag)
    #diagは対角成分の絶対値のリスト
    sortdiag = np.sort(np.array(diag))
#    print(sortdiag)
    #sortdiagはdiagを並び替えたもの
    S = np.eye(len(A))
    T = np.eye(len(A[0]))
    #S,Tはすべての要素が1の行列
    
    #diagを書き換えながらいきます
    check = True
    #このcheckはbreakをうまく使ってるからいらないかもね
    while check:
        differentpoint=None
        for i in range(0,len(A[0])):
            if abs(diag[i])!=sortdiag[i]:
                differentpoint=i
                #対角成分が異なるところ
                break
        if differentpoint==None:
            check = False
            break

        for j in range(0,len(A[0])):
            if abs(diag[differentpoint])==sortdiag[j] and abs(diag[j])!=sortdiag[j]:
                #differentpoint番目と並べ替えた後のj番目が一致かつj番目同士は異なる
#                print(diag)
                
                S2= np.eye(len(A))
                T2 = np.eye(len(A[0]))
                S2[differentpoint][j]=1
                S2[j][differentpoint]=1
                S2[differentpoint][differentpoint]=0
                S2[j][j]=0
                T2[differentpoint][j]=1
                T2[j][differentpoint]=1
                T2[differentpoint][differentpoint]=0
                T2[j][j]=0
                
                S = np.dot(np.array(S2),np.array(S))
                T = np.dot(np.array(T),np.array(T2))
                
                a=diag[differentpoint]
                b=diag[j]
                diag[differentpoint]=b
                diag[j]=a
                
    A = np.dot(np.array(S),np.array(A))
    A = np.dot(np.array(A),np.array(T))
    return [S,A,T]
#sortができた臭いぞ
#対角行列に対して対角成分を基本変形で絶対値の昇順にして返す。その際の基本変形も返す。


def modifydiag(A):
    S = np.eye(len(A))
    T = np.eye(len(A[0]))
    S2 = np.eye(len(A))
    T2 = np.eye(len(A[0]))
    check = False
    #checkがFalseのままなら全部割れる
    for i in range(0,len(A[0])-1):
        if A[i][i]!=0:
            if A[i+1][i+1]%A[i][i]!=0:
                #mod0は考えられないよー。ifの条件内なのにA[i][i]==0だったらそもそも計算できずにおかしくなる
                check = True
                breaknum = i
                break
    #breaknum番目でその次が割れない
            
    if check:
        #割り切れなかった時だよ
        i = breaknum
        beta = gcd(A[i+1][i+1],A[i][i])
        #本当はここから拡張ユークリッド互除法でσ、τを探したほうがいいよ
        for i1 in range(int(-abs(A[i+1][i+1]/beta)+1),int(abs(A[i+1][i+1])/beta)):
            for i2 in range(int(-abs(A[i][i]/beta)+1),int(abs(A[i][i])/beta)):
                sagasicheck = False
                if A[i][i]*i1+A[i+1][i+1]*i2==beta:
                    S[i][i]=i1
                    S[i][i+1]=i2
                    S[i][i+1]=-int(A[i+1][i]/beta)
                    S[i+1][i+1]=int(A[i][i]/beta)
                    
                    T[i+1][i]=1
                    #このTでi+1列をi列に足している
                    
                    sagasicheck = True
                    break
            if sagasicheck:
                break
        #(i,i)成分を(i+1,i+1)成分を使って最大公約数まで下げるパート
        
        A = np.dot(np.array(S),np.array(A))
        A = np.dot(np.array(A),np.array(T))
        
        S2[i+1][i]=-A[i+1][i]/A[i][i]
        T2[i][i+1]=-A[i][i+1]/A[i][i]
        #(i,i+1)と(i+1,i)成分を0に戻すパート
        A = np.dot(np.array(S2),np.array(A))
        A = np.dot(np.array(A),np.array(T2))
        
    return [np.dot(np.array(S2),np.array(S)),A,np.dot(np.array(T),np.array(T2)),check]
#対角成分が一個隣のやつで割り切れていたら何もせずcheck==Falseを返す。そうでない場合は一か所だけ最大公約数になるように変形する。
#計算量は怪しいがとりあえずできたことはできた

In [74]:
#カンドルチェイン複体の元を例えばC3なら[[0,1,2,-4],[1,2,0,7]]のようにあらわす。－４や７が係数

                #まずは微分をimplementする
    
def differential3(element3):
    result2 = []
    for a in element3:
#        result2.append([a[1],a[2],-a[3]])
        result2.append([a[0],a[2],a[3]])
        result2.append([a[0],a[1],-a[3]])
#        result2.append([a[1],a[2],a[3]])
        result2.append([mul(a[0],a[1]),a[2],-a[3]])
        result2.append([mul(a[0],a[2]),mul(a[1],a[2]),a[3]])
        #a[3]はC2の元の係数のつもり
#    result2 = arrangement2(result2)
    return result2
#element3はX3乗の元の最後に整数がついたもののリスト（C3の元)
#returnはC2の元

def differential4(element4):
    result3 = []
    for a in element4:
#        result3.append([a[1],a[2],a[3],-a[4]])
        result3.append([a[0],a[2],a[3],a[4]])
        result3.append([a[0],a[1],a[3],-a[4]])
        result3.append([a[0],a[1],a[2],a[4]])
#        result3.append([a[1],a[2],a[3],a[4]])
        result3.append([mul(a[0],a[1]),a[2],a[3],-a[4]])
        result3.append([mul(a[0],a[2]),mul(a[1],a[2]),a[3],a[4]])
        result3.append([mul(a[0],a[3]),mul(a[1],a[3]),mul(a[2],a[3]),-a[4]])
#    result3 = arrangement3(result3)
    return result3
#element4はX4乗の元の最後に整数がついたもののリスト（C4の元)
#returnはC3の元
                #微分の実装終わり

                #chainの元を適当な基底をとってベクトルに変換する
        
def arrang2(elem2):
    resu2 = []
    for q in list(itertools.product(Q,repeat = 2)):
        if q[0]!=q[1]:
        #ここで隣同士が一致するものは弾きます
            scalar = 0
            for a in elem2:
                if a[0] == q[0] and a[1] == q[1]:
                    scalar = scalar + a[2]
            resu2.append(scalar)
    return resu2

def arrang3(elem3):
    resu3 = []
    for q in list(itertools.product(Q,repeat = 3)):
        if q[0]!=q[1] and q[1]!=q[2]:
            scalar = 0
            for a in elem3:
                if a[0] == q[0] and a[1] == q[1] and a[2] == q[2]:
                    scalar = scalar + a[3]
            resu3.append(scalar)
    return resu3
#C2、C3の元を隣同士が一致するならば割り、forで自然に入る基底の係数を返す関数
#forで自然に入る基底とはfor q in list(itertools.product(Q,repeat = 2or3))のqたちのこと
#elem2,elem3はC2、C3の元でreturnはC2、C3を割ったのちの次元のベクトル

                #適当な基底をとってベクトルで表示する終わり
                #もうちょい増やしたい
        
        
                #ｄのKernelの基底とかImageの基底表示とか
                      
            
def Kerneld3(Q):
    Ker=[]
    #d3のKernelの生成元からなるリストにしたい
    imd3mat = []
    for q in list(itertools.product(Q,repeat = 3)):
        if q[0]!=q[1] and q[1]!=q[2]:
            q = list(q)
            q.append(1)
            imd3mat.append(arrang2(differential3([q])))            
    #C3の自然な基底をd3で送ったものをC2の基底でベクトル表示したもの
    A = imd3mat
    S =  Smithnormalform(A)[0]
    #これのi行目はC3の元を自然な基底でベクトル表示したもの、d3で送るとSNFのi行目に飛ぶ
    SNF = Smithnormalform(A)[1]
    
    imagestart = 0
    for i in range(0,len(SNF[0])):
        if SNF[i][i]!=0:
            break
        imagestart+=1
    #imagestartからlen(A[0])-1までの基底はKernelの元じゃない
    
    for i in range(0,len(A)):
        if i<imagestart or len(A[0])<=i:
            Ker.append(S[i])
    #C3の標準的でない基底たちSの中からKerの元でないものを除いていきます

    return Ker
#d3のKerをC3の自然な基底による表示で返す



#階段行列化の関数も用意したいね
#用意しました
def uesannkakuka(A):
    #AはKerneld3のdim行でC3のdim列であることを想定
    #Aはnumpyのarrayであることを想定しています
    
    S = np.eye(len(A))
    
    kaidannzure = 0
    for t in range(0,min(len(A),len(A[0]))):
    #tは何列目を省くか
        check = False
        while check==False:
            check=retuhabuki(A,t+kaidannzure,t)[2]
            #t列目がきれいに省けたらTrueになる
            S=np.dot(retuhabuki(A,t+kaidannzure,t)[0],S)
            A=retuhabuki(A,t+kaidannzure,t)[1]
            if check==None:
                kaidannzure += 1
                check = False
            #print(kaidannzure)
            
        #これがt列目を省く部分
        #print(t)
        #print(S)
        #print(A)
    return [S,A]
#行列に対して左から順に整理していって上三角化する関数。その際の行基本変形はSとして返す。
                    
    #上三角化だけじゃ不十分だぞーーーーーーーーーー
    #ちゃんとランク計算できるところまで整理しやがれーーーーーーーーーーー
    #階段行列化に成功しました



def retuhabuki(A,t,s):
    #Aのt列目をs行より下の要素で消したい
    #階段行列を作るために省きます。そのためt列目を省く際にs行より上の行には干渉しないように作ります。関数名と中身に違和感があるけど許してね。
    S = np.eye(len(A))
    #左からかける行列SとA自体を弄って返す
    
    jt = s
        #t列目のjt行がpivotのある場所。今階段行列にしたいからpivotはs行より下に探すよ
        
    jtkari = 0
    ajtt = 0
    #pivotの値にする予定
    #A[jt]がpivot行
    for v in A:
        if abs(v[t]) == 1 and s<=jtkari:
            ajtt=v[t]
            jt = jtkari
            break
        elif v[t]!=0 and s<=jtkari:
            if ajtt==0:
                ajtt = v[t]
                jt = jtkari
            else:
                if abs(v[t])<abs(ajtt):
                    ajtt = v[t]
                    jt = jtkari
        jtkari += 1
            #jtは最後まで行った場合はlen(A)となることに注意
        #ここまでで大体step1,pivotの場所を指定している
        #ajttにはpivotでの値が、jtは一番絶対値の小さいところが入る。ただし、ajtt==0の時はjt=tに勝手になるから参考にならない
        
    #print(jt)
    #print(jtkari)
    
    if ajtt == 0:
        #t列目のs行よりしたが全部0だったよ
        return [S,A,None]
    else:
    
        pivotcheck = True
        #ajttでt列目が割れるかどうか判定
        kt=0
        #割り切れなかった時の割れなかった行数
        for v in A:
            if v[t]%A[jt][t]!=0:
                pivotcheck = False
                break
            kt+=1
        #print(kt)
        
        if pivotcheck:
        #割り切れたとき
            for k in range(0,len(A)):
                if k!=jt:
                    #kがpivot行目ではないよ
                    S[k][jt] = -A[k][t]/A[jt][t]
                    #jt行目でk行を消す
            S2 = np.eye(len(A))
            if s!=jt:
                #もしpivot行（jt行）がs行目でなければ
                S2[s][s]=0
                S2[jt][s]=1
                S2[s][jt]=1 
                S2[jt][jt]=0
                

            S = np.dot(np.array(S2),np.array(S))
            #もしs!=jtならjt行目をs行目と入れ替える
            A = np.dot(np.array(S),np.array(A))
    
        else:
            #割り切れなかった時
            #A[kt][t]とA[jt][t]で最大公約数云々をする
            #print([A[kt][t],A[jt][t]])
            beta = gcd(A[kt][t],A[jt][t])
            #print(beta)
            #本当はここから拡張ユークリッド互除法でσ、τを探したほうがいいよ
            for i1 in range(int(-abs(A[jt][t]/beta)+1),int(abs(A[jt][t])/beta)):
                for i2 in range(int(-abs(A[kt][t]/beta)+1),int(abs(A[kt][t])/beta)):
                    sagasicheck = False
                    if A[kt][t]*i1+A[jt][t]*i2==beta:
                        S[jt][jt]=i2
                        S[kt][jt]=i1
                        S[jt][kt]=-int(A[kt][t]/beta)
                        S[kt][kt]=int(A[jt][t]/beta)
                        sagasicheck = True
                        break
                if sagasicheck:
                    break
            A = np.dot(np.array(S),np.array(A))
            
        return [S,A,pivotcheck]
#全部割り切れるなら（pivotcheck==True）Aのt列目を省いてjt行目をt行目に移動する。この時右からSをかけてる
#全部割れないなら（pivotcheck==False）Aのjt行目を少し小さくして返す。この時右からSをかけてる
#t列目のs行より下が全て0の時はretuhabuki(A,t,s)[2]==Noneとなるように修正しました。階段行列にしたいから
#Smithalgoが平気ゆえこれも大丈夫そう
        


def Imaged4(Q):
    imd4repbykerd3 = []
    #C4の自然な生成元の微分の行き先をC3のKerneld3(Q)で表される基底でベクトル表示したものにしたい
    #このImaged4はsurface knotの不変量として求める場合を想定して作られている。一般のFxの値として求める分にはそのままforで定まるC3の自然な基底で構わない
    imd4mat = []
    for q in list(itertools.product(Q,repeat = 4)):
        if q[0]!=q[1] and q[1]!=q[2] and q[2]!=q[3]:
            q = list(q)
            q.append(1)
            imd4mat.append(arrang3(differential4([q])))
    #C4の自然な生成元の微分の行き先をC3の自然な基底でベクトル表示したもの
    for v in imd4mat:
        #vをKernelの生成元表示に切り替える。生成元は上三角で与えられていることを仮定
        vdash=[]
        for w in uesannkakuka(Kerneld3(Q))[1]:
            colnum = 0
            for a in w:
                if a != 0:
                    vdash.append(v[colnum]/a)
                    v = np.array(v)-np.inner(np.array(w),v[colnum]/a)
#                    print(v)
#                    print(colnum)
#                    print(vdash)
                    #vはw成分をv[colnum]/aだけ持っている
                    break
                colnum+=1
        imd4repbykerd3.append(vdash)
    return imd4repbykerd3
#returnはC4の自然な生成元を微分で飛ばしたものをKerneld3の基底でベクトル表示したもの
#ただしKerneld3の基底は上三角行列になっていることを仮定している。全然できていないですよ。Kerneld3の部分はね



def Imaged4forpseudofunctor(Q):
    imd4repbykerd3 = []
    #C4の自然な生成元の微分の行き先をC3のKerneld3(Q)で表される基底でベクトル表示したものにしたい
    #このImaged4はsurface braidの不変量として求める場合を想定して作られている。つまりFxの値ということ。
    imd4mat = []
    for q in list(itertools.product(Q,repeat = 4)):
        if q[0]!=q[1] and q[1]!=q[2] and q[2]!=q[3]:
            q = list(q)
            q.append(1)
            imd4mat.append(arrang3(differential4([q])))
    #C4の自然な生成元の微分の行き先をC3の自然な基底でベクトル表示したもの
    return imd4mat









#求めたSmithnormalformに対して自然な基底でどの元に対応しているかも拾いたいよーーーーーーーーーーーー





#SmithnormalformのTのi行目はKerneld3の生成元で表示された係数ベクトル。Tは逆よりこれらもまた基底
#A[i][i]回足すとImaged4に入る、（0なら何回足してもimageには入らない）

def seiseigen(A,T,genofkerd3):
    #A,TはSmithnormalformそのもの。genofkerd3はKerd3のSNFを作る際に選んできた基底
    genofh3 = []
    #H3の生成元とその位数をつけたものにしたい
    for i in range(0,len(A[0])):
        #Aのi行目
        v = np.zeros(len(genofkerd3[0]))
        for j in range(0,len(A[0])):
            v = v + np.inner(  np.array(genofkerd3[j])  , T[i][j]   )
            
        genofh3.append([       v  , A[i][i] ])
        
    return genofh3
#H3の生成元を自然な基底で表示して、その位数との対にして並べたものが返ってきます。



#C3の元が与えられたとき（closerが存在するsurf braidの場合は全体としてはKernelの元に落ち着く）に基底で表示するパート。
#自然な基底での係数ベクトルが与えられました。これをKernelの生成元の基底からなる係数ベクトルに変えます。
#ちなみにKernelの中に入っていることは保障されていますので階段化されたKernelの基底で簡単に表せます。
#最後にSNFの基底での表示に切り替えます。


def h3hoyuzi(v,T):
    #vecは自然な基底で表現されたC3の元であることを想定。いや、Kerneld3の元であることを想定。
    #TはSNFの右のやつだよ。
    vdash=[]
    for w in uesannkakuka(Kerneld3(Q))[1]:
        colnum = 0
        for a in w:
            if a != 0:
                vdash.append(v[colnum]/a)
                v = np.array(v)-np.inner(np.array(w),v[colnum]/a)
#               print(v)
#               print(colnum)
#               print(vdash)
                #vはw成分をv[colnum]/aだけ持っている
                break
            colnum += 1
    #vdashはkerneld3基底表示のベクトル
    vinh3 = np.dot(np.array(vdash),T)
    return vinh3









In [52]:
Smith = Smithnormalform(Imaged4(Q))
Smith

[array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1., -3., -1.,  0.

In [54]:
seiseigen(Smith[1],Smith[2],Kerneld3(Q))

#これがH3の生成元だよーーーーー

[[array([ 1.,  0.,  1.,  1.,  0., -2.,  0., -1.,  1., -1.,  0.,  0.]), 1.0],
 [array([-2.,  1.,  1.,  0.,  2., -3.,  0., -1.,  0., -1., -1., -2.]), -1.0],
 [array([-4.,  0., -1.,  0.,  5., -4.,  1., -1.,  0., -1., -2., -3.]), -1.0],
 [array([ 2.,  0.,  0.,  0., -2.,  3.,  0.,  1., -1.,  1.,  1.,  1.]), 1.0],
 [array([ 1.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.]), -1.0],
 [array([ 1.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.]), -1.0],
 [array([ 2.,  0.,  1.,  0., -2.,  1.,  0.,  0.,  0.,  0.,  1.,  1.]), 1.0],
 [array([ 1.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.]), 3.0]]

In [53]:
Kerneld3(Q)

[array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]),
 array([-1.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  1.,  0.,  0.,  0.]),
 array([-1.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]),
 array([ 1.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.])]

In [77]:
for v in Kerneld3(Q):
    vec = v
    print(h3hoyuzi(vec,Smith[2]))

[1. 0. 0. 0. 0. 0. 1. 1.]
[ 0.  1. -1.  0.  0.  0.  1.  1.]
[0. 0. 0. 0. 1. 0. 1. 1.]
[0. 0. 0. 0. 0. 1. 1. 1.]
[-1.  0.  0.  1.  0.  0.  1.  1.]
[-1.  0.  1.  0.  0.  0. -1. -1.]
[ 1.  0. -1.  0.  0.  0.  1.  2.]
[ 1.  0.  0. -1.  0.  0.  0.  0.]


In [70]:
h3hoyuzi(vec,Smith[2])

array([ 1.,  0.,  0., -1.,  1., -1.,  0.,  0.])

In [12]:
for v in Kerneld3(Q):
    print(v[5])

1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [13]:

Smith = Smithnormalform(Imaged4(Q))[1]

In [15]:
A=Imaged4(Q)
np.array(A)
#Smithnormalform(A)

array([[ 1.,  0.,  0., ...,  0., -0.,  0.],
       [ 1., -1.,  0., ...,  0., -0.,  0.],
       [ 1.,  0., -1., ...,  0., -0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0., -0.,  0.],
       [ 0.,  0.,  0., ..., -1., -0.,  0.],
       [ 0.,  0.,  0., ...,  0., -0.,  0.]])

In [16]:
Smithnormalform(A)

[array([[ 1., -1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [-1.,  0.,  1., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0., -1., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
          1.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  

In [17]:
B = Smithnormalform(A)[1][35]
B

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
for i in range(0,36):
    print(Smithnormalform(A)[1][i][i])

0.0
0.0
-1.0
1.0
-1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
-1.0
-1.0
1.0
-1.0
1.0
-1.0
2.0
-2.0
2.0
-2.0


IndexError: index 28 is out of bounds for axis 0 with size 28

In [19]:
#階段行列化の関数も用意したいね
#用意しました
def uesannkakuka(A):
    #AはKerneld3のdim行でC3のdim列であることを想定
    #Aはnumpyのarrayであることを想定しています
    
    S = np.eye(len(A))
    
    kaidannzure = 0
    for t in range(0,min(len(A),len(A[0]))):
    #tは何列目を省くか
        check = False
        while check==False:
            check=retuhabuki(A,t+kaidannzure,t)[2]
            #t列目がきれいに省けたらTrueになる
            S=np.dot(retuhabuki(A,t+kaidannzure,t)[0],S)
            A=retuhabuki(A,t+kaidannzure,t)[1]
            if check==None:
                kaidannzure += 1
                check = False
            print(kaidannzure)
            
        #これがt列目を省く部分
        #print(t)
        #print(S)
        #print(A)
    return [S,A]
#行列に対して左から順に整理していって上三角化する関数。その際の行基本変形はSとして返す。
                    
    #上三角化だけじゃ不十分だぞーーーーーーーーーー
    #ちゃんとランク計算できるところまで整理しやがれーーーーーーーーーーー
    #階段行列化に成功しました



def retuhabuki(A,t,s):
    #Aのt列目をs行より下の要素で消したい
    #階段行列を作るために省きます。そのためt列目を省く際にs行より上の行には干渉しないように作ります。関数名と中身に違和感があるけど許してね。
    S = np.eye(len(A))
    #左からかける行列SとA自体を弄って返す
    
    jt = s
        #t列目のjt行がpivotのある場所。今階段行列にしたいからpivotはs行より下に探すよ
        
    jtkari = 0
    ajtt = 0
    #pivotの値にする予定
    #A[jt]がpivot行
    for v in A:
        if abs(v[t]) == 1 and s<=jtkari:
            ajtt=v[t]
            jt = jtkari
            break
        elif v[t]!=0 and s<=jtkari:
            if ajtt==0:
                ajtt = v[t]
                jt = jtkari
            else:
                if abs(v[t])<abs(ajtt):
                    ajtt = v[t]
                    jt = jtkari
        jtkari += 1
            #jtは最後まで行った場合はlen(A)となることに注意
        #ここまでで大体step1,pivotの場所を指定している
        #ajttにはpivotでの値が、jtは一番絶対値の小さいところが入る。ただし、ajtt==0の時はjt=tに勝手になるから参考にならない
        
    #print(jt)
    #print(jtkari)
    
    if ajtt == 0:
        #t列目のs行よりしたが全部0だったよ
        return [S,A,None]
    else:
    
        pivotcheck = True
        #ajttでt列目が割れるかどうか判定
        kt=0
        #割り切れなかった時の割れなかった行数
        for v in A:
            if v[t]%A[jt][t]!=0:
                pivotcheck = False
                break
            kt+=1
        #print(kt)
        
        if pivotcheck:
        #割り切れたとき
            for k in range(0,len(A)):
                if k!=jt:
                    #kがpivot行目ではないよ
                    S[k][jt] = -A[k][t]/A[jt][t]
                    #jt行目でk行を消す
            S2 = np.eye(len(A))
            if s!=jt:
                #もしpivot行（jt行）がs行目でなければ
                S2[s][s]=0
                S2[jt][s]=1
                S2[s][jt]=1 
                S2[jt][jt]=0
                

            S = np.dot(np.array(S2),np.array(S))
            #もしs!=jtならjt行目をs行目と入れ替える
            A = np.dot(np.array(S),np.array(A))
    
        else:
            #割り切れなかった時
            #A[kt][t]とA[jt][t]で最大公約数云々をする
            #print([A[kt][t],A[jt][t]])
            beta = gcd(A[kt][t],A[jt][t])
            #print(beta)
            #本当はここから拡張ユークリッド互除法でσ、τを探したほうがいいよ
            for i1 in range(int(-abs(A[jt][t]/beta)+1),int(abs(A[jt][t])/beta)):
                for i2 in range(int(-abs(A[kt][t]/beta)+1),int(abs(A[kt][t])/beta)):
                    sagasicheck = False
                    if A[kt][t]*i1+A[jt][t]*i2==beta:
                        S[jt][jt]=i2
                        S[kt][jt]=i1
                        S[jt][kt]=-int(A[kt][t]/beta)
                        S[kt][kt]=int(A[jt][t]/beta)
                        sagasicheck = True
                        break
                if sagasicheck:
                    break
            A = np.dot(np.array(S),np.array(A))
            
        return [S,A,pivotcheck]
#全部割り切れるなら（pivotcheck==True）Aのt列目を省いてjt行目をt行目に移動する。この時右からSをかけてる
#全部割れないなら（pivotcheck==False）Aのjt行目を少し小さくして返す。この時右からSをかけてる
#t列目のs行より下が全て0の時はretuhabuki(A,t,s)[2]==Noneとなるように修正しました。階段行列にしたいから
#Smithalgoが平気ゆえこれも大丈夫そう